In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
spark = SparkSession.builder.appName('fireCall').getOrCreate()

In [0]:
fire_schema = StructType([StructField('CallNumber', IntegerType(), True),
                StructField('UnitID', StringType(), True),
                StructField('IncidentNumber', IntegerType(), True),
                StructField('CallType', StringType(), True),                  
                StructField('CallDate', StringType(), True),      
                StructField('WatchDate', StringType(), True),
                StructField('CallFinalDisposition', StringType(), True),
                StructField('AvailableDtTm', StringType(), True),
                StructField('Address', StringType(), True),       
                StructField('City', StringType(), True),       
                StructField('Zipcode', IntegerType(), True),       
                StructField('Battalion', StringType(), True),                 
                StructField('StationArea', StringType(), True),       
                StructField('Box', StringType(), True),       
                StructField('OriginalPriority', StringType(), True),       
                StructField('Priority', StringType(), True),       
                StructField('FinalPriority', IntegerType(), True),       
                StructField('ALSUnit', BooleanType(), True),       
                StructField('CallTypeGroup', StringType(), True),
                StructField('NumAlarms', IntegerType(), True),
                StructField('UnitType', StringType(), True),
                StructField('UnitSequenceInCallDispatch', IntegerType(), True),
                StructField('FirePreventionDistrict', StringType(), True),
                StructField('SupervisorDistrict', StringType(), True),
                StructField('Neighborhood', StringType(), True),
                StructField('Location', StringType(), True),
                StructField('RowID', StringType(), True),
                StructField('Delay', FloatType(), True)])

In [0]:
fire_df = spark.read.csv("/databricks-datasets/learning-spark-v2/sf-fire/sf-fire-calls.csv",header=True,schema=fire_schema)

In [0]:
fire_df.head()

Out[4]: Row(CallNumber=20110014, UnitID='M29', IncidentNumber=2003234, CallType='Medical Incident', CallDate='01/11/2002', WatchDate='01/10/2002', CallFinalDisposition='Other', AvailableDtTm='01/11/2002 01:58:43 AM', Address='10TH ST/MARKET ST', City='SF', Zipcode=94103, Battalion='B02', StationArea='36', Box='2338', OriginalPriority='1', Priority='1', FinalPriority=2, ALSUnit=True, CallTypeGroup=None, NumAlarms=1, UnitType='MEDIC', UnitSequenceInCallDispatch=1, FirePreventionDistrict='2', SupervisorDistrict='6', Neighborhood='Tenderloin', Location='(37.7765408927183, -122.417501464907)', RowID='020110014-M29', Delay=5.233333110809326)

In [0]:
fire_df.columns

Out[7]: ['CallNumber',
 'UnitID',
 'IncidentNumber',
 'CallType',
 'CallDate',
 'WatchDate',
 'CallFinalDisposition',
 'AvailableDtTm',
 'Address',
 'City',
 'Zipcode',
 'Battalion',
 'StationArea',
 'Box',
 'OriginalPriority',
 'Priority',
 'FinalPriority',
 'ALSUnit',
 'CallTypeGroup',
 'NumAlarms',
 'UnitType',
 'UnitSequenceInCallDispatch',
 'FirePreventionDistrict',
 'SupervisorDistrict',
 'Neighborhood',
 'Location',
 'RowID',
 'Delay']

In [0]:
few_fire_df=fire_df.select("IncidentNumber","AvailableDtTm","CallTypeGroup").where(fire_df["CallTypeGroup"]!="Medical Incident")
few_fire_df.show(5,truncate=False)

+--------------+----------------------+--------------------+
IncidentNumber|AvailableDtTm |CallTypeGroup |
+--------------+----------------------+--------------------+
11077699 |08/23/2011 09:26:39 PM|Non Life-threatening|
11077699 |08/23/2011 08:43:19 PM|Non Life-threatening|
12000005 |01/01/2012 01:00:43 AM|Non Life-threatening|
12000002 |01/01/2012 12:20:19 AM|Fire |
12000003 |null |Fire |
+--------------+----------------------+--------------------+
only showing top 5 rows

In [0]:
# In Python, return number of distinct types of calls using countDistinct()
from pyspark.sql.functions import *
distCallType2 = fire_df.select('CallType').where(col('CallType').isNotNull()).agg(countDistinct('CallType'))
distCallType2.show()

+---------------+
count(CallType)|
+---------------+
 32|
+---------------+

In [0]:
# In Python, filter for only distinct non-null CallTypes from all the rows
fire_df.select("CallType").where(col('CallType').isNotNull()).distinct().show(10)

+--------------------+
 CallType|
+--------------------+
 Alarms|
Odor (Strange / U...|
Citizen Assist / ...|
 Vehicle Fire|
 Other|
 Outside Fire|
 Electrical Hazard|
 Structure Fire|
 Medical Incident|
 Fuel Spill|
+--------------------+
only showing top 10 rows

In [0]:
new_fire_df=fire_df.withColumnRenamed('Delay','ResponseDelayedinMin')
new_fire_df.select('ResponseDelayedinMin').where(col('ResponseDelayedinMin') > 5).show(5,truncate=False)

+--------------------+
ResponseDelayedinMin|
+--------------------+
5.233333 |
6.9333334 |
6.116667 |
7.85 |
77.333336 |
+--------------------+
only showing top 5 rows

In [0]:
fire_ts_df = new_fire_df.withColumn('IncidentDate',to_timestamp(col('CallDate'),'MM/dd/yyyy')).drop('CallDate')\
.withColumn('OnWatchDate',to_timestamp(col('WatchDate'),'MM/dd/yyyy')).drop('WatchDate')\
.withColumn('AvailableDtTS',to_timestamp(col('AvailableDtTm'),'MM/dd/yyyy hh:mm:ss a')).drop('AvailableDtTm')
fire_ts_df.select('IncidentDate','OnWatchDate','AvailableDtTS').show(5,truncate=False)    

+-------------------+-------------------+-------------------+
IncidentDate |OnWatchDate |AvailableDtTS |
+-------------------+-------------------+-------------------+
2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:58:43|
2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 02:10:17|
2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:47:00|
2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:51:54|
2002-01-11 00:00:00|2002-01-10 00:00:00|2002-01-11 01:47:00|
+-------------------+-------------------+-------------------+
only showing top 5 rows

In [0]:
fire_ts_df.select(year('IncidentDate')).distinct().orderBy(year(col('IncidentDate'))).show()

+------------------+
year(IncidentDate)|
+------------------+
 2000|
 2001|
 2002|
 2003|
 2004|
 2005|
 2006|
 2007|
 2008|
 2009|
 2010|
 2011|
 2012|
 2013|
 2014|
 2015|
 2016|
 2017|
 2018|
+------------------+

In [0]:
# what were the most common types of fire calls?
fire_ts_df.select('CallType').where(col('CallType').isNotNull()).groupBy('CallType').count().orderBy('count', ascending=False).show(10,truncate=False)

+-------------------------------+-------+
CallType |count |
+-------------------------------+-------+
Medical Incident |2843475|
Structure Fire |578998 |
Alarms |483518 |
Traffic Collision |175507 |
Citizen Assist / Service Call |65360 |
Other |56961 |
Outside Fire |51603 |
Vehicle Fire |20939 |
Water Rescue |20037 |
Gas Leak (Natural and LP Gases)|17284 |
+-------------------------------+-------+
only showing top 10 rows

In [0]:
#compute the sum of alarms, the average response time, and the minimum and maximum response times to all fire calls
import pyspark.sql.functions as F
fire_ts_df.select(F.sum('NumAlarms'),F.avg('ResponseDelayedinMin'),F.min('ResponseDelayedinMin'),F.max('ResponseDelayedinMin')).show()

+--------------+-------------------------+-------------------------+-------------------------+
sum(NumAlarms)|avg(ResponseDelayedinMin)|min(ResponseDelayedinMin)|max(ResponseDelayedinMin)|
+--------------+-------------------------+-------------------------+-------------------------+
 4403441| 3.902170335891614| 0.016666668| 1879.6167|
+--------------+-------------------------+-------------------------+-------------------------+